# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [4]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('D:\DataScience-Python3\emails\spam', 'spam'))
data = data.append(dataFrameFromDirectory('D:\DataScience-Python3\emails\ham', 'ham'))


Let's have a look at that DataFrame:

In [5]:
data.head()

,class,message
D:\DataScience-Python3\emails\spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
D:\DataScience-Python3\emails\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
D:\DataScience-Python3\emails\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
D:\DataScience-Python3\emails\spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
D:\DataScience-Python3\emails\spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [34]:
data=data.sample(frac=1)

vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
targets = data['class'].values

trainX = counts[:-100]
testX = counts[-100:]

trainY = targets[:-100]
testY = targets[-100:]

classifier = MultinomialNB()

classifier.fit(trainX, trainY)
data.head()

,class,message
D:\DataScience-Python3\emails\ham\00622.80ded6eb7b17a2bb45d502b85244dad1,ham,"On Wednesday 18 September 2002 06:47 am, Rober..."
D:\DataScience-Python3\emails\ham\02308.0fa0a0b2d072a7d8f863c0fe5b34045e,ham,URL: http://boingboing.net/#85528777\n\nDate: ...
D:\DataScience-Python3\emails\ham\00718.1fc4cf16dbe6d0395c754671b24782e1,ham,\n\n[Destined to be a new reality TV Show?]\n\...
D:\DataScience-Python3\emails\ham\01140.71d7100b58f00faaf258b49150c6f54a,ham,In article <E17tS0T-0001zD-00@rds059> you writ...
D:\DataScience-Python3\emails\spam\00051.fd20658f0e586d1f27f9396401f4981c,spam,Help wanted. We are a 14 year old fortune 500...


Let's try it out:

In [35]:
examples = ['Free viagra ', "Hi bob, how about meeting tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'],
      dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [36]:
predictions1 = classifier.predict(testX)
predictions1

array(['ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham'],
      dtype='<U4')

In [37]:
testY

array(['ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham'], dtype=object)

In [39]:
count_int =0
for ar in range(len(predictions1)):
    if (predictions1[ar] == testY[ar]):
        count_int += 1
        
accr = count_int/100

In [40]:
accr

0.96